In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os



In [2]:
PROJECT_ROOT = Path(r"C:\Users\joshu\OneDrive\Desktop\Projects and Analysis 2\crime-housing-affordability")
os.chdir(PROJECT_ROOT)


In [3]:
from config.financial_assumptions import (
    MONTHLY_GROSS_INCOME,
    FEDERAL_EFFECTIVE_TAX,
    PAYROLL_TAX,
    NON_HOUSING_MONTHLY,
    MONTHLY_SAVINGS_TARGET,
    HOUSING_COMFORTABLE_RATIO,
    HOUSING_STRETCHED_RATIO,
    MORTGAGE_RATE,
    MORTGAGE_TERM_YEARS,
    DOWN_PAYMENT_RATE,
    PROPERTY_TAX_RATE,
    HOME_INSURANCE_MONTHLY
)


In [4]:
zhvi_raw = pd.read_csv("data/raw/zillow/zillow_zhvi_sfr_midtier_metro.csv")
zori_raw = pd.read_csv("data/raw/zillow/zillow_zori_rent_metro.csv")

print("ZHVI raw shape:", zhvi_raw.shape)
print("ZORI raw shape:", zori_raw.shape)



ZHVI raw shape: (895, 316)
ZORI raw shape: (556, 136)


In [5]:
id_cols = ["RegionID", "SizeRank", "RegionName", "RegionType", "StateName"]

zhvi_long = zhvi_raw.melt(id_vars=id_cols, var_name="date", value_name="home_value")
zori_long = zori_raw.melt(id_vars=id_cols, var_name="date", value_name="rent")

zhvi_long["date"] = pd.to_datetime(zhvi_long["date"])
zori_long["date"] = pd.to_datetime(zori_long["date"])

print("ZHVI long:", zhvi_long.shape)
print("ZORI long:", zori_long.shape)


ZHVI long: (278345, 7)
ZORI long: (72836, 7)


In [6]:
latest_zhvi = zhvi_long["date"].max()
latest_zori = zori_long["date"].max()
latest_date = min(latest_zhvi, latest_zori)

print("Latest ZHVI:", latest_zhvi)
print("Latest ZORI:", latest_zori)
print("Using latest_date:", latest_date)


Latest ZHVI: 2025-11-30 00:00:00
Latest ZORI: 2025-11-30 00:00:00
Using latest_date: 2025-11-30 00:00:00


In [7]:
zhvi_latest = zhvi_long[zhvi_long["date"] == latest_date].copy()
zori_latest = zori_long[zori_long["date"] == latest_date].copy()

zhvi_latest.to_csv("data/processed/zhvi_latest.csv", index=False)
zori_latest.to_csv("data/processed/zori_latest.csv", index=False)

print("Saved processed Zillow latest files.")
print("ZHVI latest shape:", zhvi_latest.shape)
print("ZORI latest shape:", zori_latest.shape)


Saved processed Zillow latest files.
ZHVI latest shape: (895, 7)
ZORI latest shape: (556, 7)


In [8]:
population = pd.read_csv("data/processed/population_2024_metro.csv")
print("Population shape:", population.shape)
population.head()


Population shape: (393, 2)


,metro_name,population_2024
0,".Abilene, TX",184278.0
1,".Akron, OH",702209.0
2,".Albany, GA",145451.0
3,".Albany, OR",132474.0
4,".Albany-Schenectady-Troy, NY",913485.0


In [9]:
housing_latest = zhvi_latest.merge(
    zori_latest,
    on=["RegionID", "RegionName", "RegionType", "StateName", "SizeRank", "date"],
    how="inner"
)

print("housing_latest shape:", housing_latest.shape)
housing_latest[["RegionName", "StateName", "home_value", "rent"]].head()


housing_latest shape: (546, 8)


,RegionName,StateName,home_value,rent
0,United States,NaN,361045.029486,2203.197513
1,"New York, NY",NY,721778.104533,3499.398403
2,"Los Angeles, CA",CA,990630.088957,4318.329317
3,"Chicago, IL",IL,354696.667711,2395.615001
4,"Dallas, TX",TX,362160.794805,2311.052245


In [10]:
df = housing_latest.merge(
    population,
    left_on="RegionName",
    right_on="metro_name",
    how="left"
)

print("Merged df shape:", df.shape)
print("Population missing:", df["population_2024"].isna().sum())
df.head()


Merged df shape: (546, 10)
Population missing: 546


,RegionID,SizeRank,RegionName,RegionType,StateName,date,home_value,rent,metro_name,population_2024
0,102001,0,United States,country,NaN,2025-11-30,361045.029486,2203.197513,NaN,NaN
1,394913,1,"New York, NY",msa,NY,2025-11-30,721778.104533,3499.398403,NaN,NaN
2,753899,2,"Los Angeles, CA",msa,CA,2025-11-30,990630.088957,4318.329317,NaN,NaN
3,394463,3,"Chicago, IL",msa,IL,2025-11-30,354696.667711,2395.615001,NaN,NaN
4,394514,4,"Dallas, TX",msa,TX,2025-11-30,362160.794805,2311.052245,NaN,NaN


In [11]:
df["monthly_net_income"] = MONTHLY_GROSS_INCOME * (1 - FEDERAL_EFFECTIVE_TAX - PAYROLL_TAX)

df["non_housing_monthly"] = NON_HOUSING_MONTHLY
df["savings_target"] = MONTHLY_SAVINGS_TARGET

df[["monthly_net_income", "non_housing_monthly", "savings_target"]].head()


,monthly_net_income,non_housing_monthly,savings_target
0,6195.833333,2340,833.333333
1,6195.833333,2340,833.333333
2,6195.833333,2340,833.333333
3,6195.833333,2340,833.333333
4,6195.833333,2340,833.333333


In [12]:
df["rent_burden"] = df["rent"] / MONTHLY_GROSS_INCOME

df["rent_leftover"] = (
    df["monthly_net_income"]
    - df["rent"]
    - df["non_housing_monthly"]
    - df["savings_target"]
)

df["rent_status"] = np.select(
    [
        df["rent_burden"] <= HOUSING_COMFORTABLE_RATIO,
        df["rent_burden"] <= HOUSING_STRETCHED_RATIO
    ],
    ["Comfortable", "Stretched"],
    default="Not Viable"
)

df["rent_status"].value_counts(dropna=False)


rent_status
Comfortable    469
Stretched       52
Not Viable      25
Name: count, dtype: int64

In [13]:
loan_amount = df["home_value"] * (1 - DOWN_PAYMENT_RATE)
monthly_rate = MORTGAGE_RATE / 12
n_payments = MORTGAGE_TERM_YEARS * 12

df["mortgage_payment_pi"] = (
    loan_amount
    * (monthly_rate * (1 + monthly_rate) ** n_payments)
    / ((1 + monthly_rate) ** n_payments - 1)
)

df["property_tax_monthly"] = df["home_value"] * PROPERTY_TAX_RATE / 12

df["mortgage_total_monthly"] = (
    df["mortgage_payment_pi"]
    + df["property_tax_monthly"]
    + HOME_INSURANCE_MONTHLY
)

df["mortgage_burden"] = df["mortgage_total_monthly"] / MONTHLY_GROSS_INCOME

df["mortgage_leftover"] = (
    df["monthly_net_income"]
    - df["mortgage_total_monthly"]
    - df["non_housing_monthly"]
    - df["savings_target"]
)

df["mortgage_status"] = np.select(
    [
        df["mortgage_burden"] <= HOUSING_COMFORTABLE_RATIO,
        df["mortgage_burden"] <= HOUSING_STRETCHED_RATIO
    ],
    ["Comfortable", "Stretched"],
    default="Not Viable"
)

df["mortgage_status"].value_counts(dropna=False)


mortgage_status
Comfortable    399
Stretched       85
Not Viable      62
Name: count, dtype: int64

In [14]:
# ==============================
# Required income (break-even): leftover = 0
# ==============================

effective_tax_rate = FEDERAL_EFFECTIVE_TAX + PAYROLL_TAX

# Renting required income
df["required_gross_monthly_rent"] = (
    df["rent"]
    + NON_HOUSING_MONTHLY
    + MONTHLY_SAVINGS_TARGET
) / (1 - effective_tax_rate)

df["required_annual_income_rent"] = (df["required_gross_monthly_rent"] * 12).round(0)

# Buying required income
df["required_gross_monthly_buy"] = (
    df["mortgage_total_monthly"]
    + NON_HOUSING_MONTHLY
    + MONTHLY_SAVINGS_TARGET
) / (1 - effective_tax_rate)

df["required_annual_income_buy"] = (df["required_gross_monthly_buy"] * 12).round(0)

print("Created required income columns.")
print(df[["RegionName", "StateName", "required_annual_income_rent", "required_annual_income_buy"]].head())


Created required income columns.
        RegionName StateName  required_annual_income_rent  \
0    United States       NaN                      86777.0   
1     New York, NY        NY                     107697.0   
2  Los Angeles, CA        CA                     120915.0   
3      Chicago, IL        IL                      89882.0   
4       Dallas, TX        TX                      88517.0   

   required_annual_income_buy  
0                     88931.0  
1                    124193.0  
2                    150474.0  
3                     88310.0  
4                     89040.0  


In [15]:
print([c for c in df.columns if "required" in c.lower()])


['required_gross_monthly_rent', 'required_annual_income_rent', 'required_gross_monthly_buy', 'required_annual_income_buy']


In [16]:
zhvi_latest.columns


Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'date',
       'home_value'],
      dtype='object')

In [17]:
# ==============================
# Clean numeric types + rounding
# ==============================

dollar_cols = [
    "home_value",
    "rent",
    "rent_leftover",
    "mortgage_total_monthly",
    "mortgage_leftover"
]

ratio_cols = [
    "rent_burden",
    "mortgage_burden"
]

# Force numeric (defensive)
for col in dollar_cols + ratio_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Round
df[dollar_cols] = df[dollar_cols].round(0)
df[ratio_cols] = df[ratio_cols].round(3)


In [18]:
needed = [
    "required_gross_monthly_buy",
    "required_annual_income_buy",
    "mortgage_total_monthly",
    "rent",
]

missing = [c for c in needed if c not in df.columns]
print("Missing columns:", missing)

# optional: show any columns containing "required"
print([c for c in df.columns if "required" in c.lower()])


Missing columns: []
['required_gross_monthly_rent', 'required_annual_income_rent', 'required_gross_monthly_buy', 'required_annual_income_buy']


In [27]:
# ==============================
# Final export: metro affordability baseline
# ==============================

export_cols = [
    "RegionID",
    "RegionName",
    "StateName",
    "home_value",
    "rent",
    "rent_burden",
    "rent_leftover",
    "rent_status",
    "required_annual_income_rent"
    "mortgage_total_monthly",
    "mortgage_burden",
    "mortgage_leftover",
    "mortgage_status",
    "required_annual_income_buy"
]

final_df = df[export_cols].copy()

final_df.to_csv(
    "data/processed/metro_affordability_baseline.csv",
    index=False
)

print("Exported:", final_df.shape)
print("Saved to data/processed/metro_affordability_baseline.csv")


KeyError: "['required_annual_income_rentmortgage_total_monthly'] not in index"

In [26]:
final_df.shape, final_df.columns.tolist()


((546, 13),
 ['RegionID',
  'RegionName',
  'StateName',
  'home_value',
  'rent',
  'rent_burden',
  'rent_leftover',
  'rent_status',
  'mortgage_total_monthly',
  'mortgage_burden',
  'mortgage_leftover',
  'mortgage_status',
  'required_annual_income_buy'])

In [20]:
final_df.head()


,RegionID,RegionName,StateName,home_value,rent,rent_burden,rent_leftover,rent_status,mortgage_total_monthly,mortgage_burden,mortgage_leftover,mortgage_status,required_annual_income_buy
0,102001,United States,NaN,361045.0,2203.0,0.264,819.0,Comfortable,2337.0,0.280,686.0,Comfortable,88931.0
1,394913,"New York, NY",NY,721778.0,3499.0,0.420,-477.0,Not Viable,4521.0,0.543,-1499.0,Not Viable,124193.0
2,753899,"Los Angeles, CA",CA,990630.0,4318.0,0.518,-1296.0,Not Viable,6150.0,0.738,-3127.0,Not Viable,150474.0
3,394463,"Chicago, IL",IL,354697.0,2396.0,0.287,627.0,Comfortable,2298.0,0.276,724.0,Comfortable,88310.0
4,394514,"Dallas, TX",TX,362161.0,2311.0,0.277,711.0,Comfortable,2343.0,0.281,679.0,Comfortable,89040.0


In [21]:
final_df["rent_status"].value_counts(), final_df["mortgage_status"].value_counts()


(rent_status
 Comfortable    469
 Stretched       52
 Not Viable      25
 Name: count, dtype: int64,
 mortgage_status
 Comfortable    399
 Stretched       85
 Not Viable      62
 Name: count, dtype: int64)

In [22]:
effective_tax_rate = FEDERAL_EFFECTIVE_TAX + PAYROLL_TAX


In [23]:
df["required_gross_monthly_rent"] = (
    df["rent"]
    + NON_HOUSING_MONTHLY
    + MONTHLY_SAVINGS_TARGET
) / (1 - effective_tax_rate)

df["required_annual_income_rent"] = (
    df["required_gross_monthly_rent"] * 12
).round(0)


In [24]:
df["required_gross_monthly_buy"] = (
    df["mortgage_total_monthly"]
    + NON_HOUSING_MONTHLY
    + MONTHLY_SAVINGS_TARGET
) / (1 - effective_tax_rate)

df["required_annual_income_buy"] = (
    df["required_gross_monthly_buy"] * 12
).round(0)


In [25]:
# ==============================
# Required income to break even (leftover = 0)
# ==============================

effective_tax_rate = FEDERAL_EFFECTIVE_TAX + PAYROLL_TAX

# Renting
df["required_gross_monthly_rent"] = (
    df["rent"]
    + NON_HOUSING_MONTHLY
    + MONTHLY_SAVINGS_TARGET
) / (1 - effective_tax_rate)

df["required_annual_income_rent"] = (
    df["required_gross_monthly_rent"] * 12
).round(0)

# Buying
df["required_gross_monthly_buy"] = (
    df["mortgage_total_monthly"]
    + NON_HOUSING_MONTHLY
    + MONTHLY_SAVINGS_TARGET
) / (1 - effective_tax_rate)

df["required_annual_income_buy"] = (
    df["required_gross_monthly_buy"] * 12
).round(0)
